In [144]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer



In [145]:
df = pd.read_csv('data.csv')
print(df.shape)
df.head()

(705, 13)


,Student_ID,Age,Gender,Academic_Level,Country,Avg_Daily_Usage_Hours,Most_Used_Platform,Affects_Academic_Performance,Sleep_Hours_Per_Night,Mental_Health_Score,Relationship_Status,Conflicts_Over_Social_Media,Addicted_Score
0,1,19,Female,Undergraduate,Bangladesh,5.2,Instagram,Yes,6.5,6,In Relationship,3,8
1,2,22,Male,Graduate,India,2.1,Twitter,No,7.5,8,Single,0,3
2,3,20,Female,Undergraduate,USA,6.0,TikTok,Yes,5.0,5,Complicated,4,9
3,4,18,Male,High School,UK,3.0,YouTube,No,7.0,7,Single,1,4
4,5,21,Male,Graduate,Canada,4.5,Facebook,Yes,6.0,6,In Relationship,2,7


In [146]:
df['Addicted_Score'].unique()

array([8, 3, 9, 4, 7, 2, 5, 6])

In [147]:
def classify_addiction(score):
    if score <= 3:
        return 'Low'
    elif score <= 6:
        return 'Moderate'
    elif score <= 8:
        return 'High'
    else:
        return 'Severe'

df['Addiction_Level'] = df['Addicted_Score'].apply(classify_addiction)

df.head()

,Student_ID,Age,Gender,Academic_Level,Country,Avg_Daily_Usage_Hours,Most_Used_Platform,Affects_Academic_Performance,Sleep_Hours_Per_Night,Mental_Health_Score,Relationship_Status,Conflicts_Over_Social_Media,Addicted_Score,Addiction_Level
0,1,19,Female,Undergraduate,Bangladesh,5.2,Instagram,Yes,6.5,6,In Relationship,3,8,High
1,2,22,Male,Graduate,India,2.1,Twitter,No,7.5,8,Single,0,3,Low
2,3,20,Female,Undergraduate,USA,6.0,TikTok,Yes,5.0,5,Complicated,4,9,Severe
3,4,18,Male,High School,UK,3.0,YouTube,No,7.0,7,Single,1,4,Moderate
4,5,21,Male,Graduate,Canada,4.5,Facebook,Yes,6.0,6,In Relationship,2,7,High


In [148]:
df_model = df.drop(columns=['Student_ID', 'Addicted_Score', 'Country']) 
df_encoded = df_model.copy() 
label_encoders = {}

for col in df_encoded.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le

df_encoded.head()


,Age,Gender,Academic_Level,Avg_Daily_Usage_Hours,Most_Used_Platform,Affects_Academic_Performance,Sleep_Hours_Per_Night,Mental_Health_Score,Relationship_Status,Conflicts_Over_Social_Media,Addiction_Level
0,19,0,2,5.2,1,1,6.5,6,1,3,0
1,22,1,0,2.1,7,0,7.5,8,2,0,1
2,20,0,2,6.0,6,1,5.0,5,0,4,3
3,18,1,1,3.0,11,0,7.0,7,2,1,2
4,21,1,0,4.5,0,1,6.0,6,1,2,0


In [149]:
for col, le in label_encoders.items():
    print(f"\nColumn: {col}")
    for i, class_ in enumerate(le.classes_):
        print(f"  {i} = {class_}")


Column: Gender
  0 = Female
  1 = Male

Column: Academic_Level
  0 = Graduate
  1 = High School
  2 = Undergraduate

Column: Most_Used_Platform
  0 = Facebook
  1 = Instagram
  2 = KakaoTalk
  3 = LINE
  4 = LinkedIn
  5 = Snapchat
  6 = TikTok
  7 = Twitter
  8 = VKontakte
  9 = WeChat
  10 = WhatsApp
  11 = YouTube

Column: Affects_Academic_Performance
  0 = No
  1 = Yes

Column: Relationship_Status
  0 = Complicated
  1 = In Relationship
  2 = Single

Column: Addiction_Level
  0 = High
  1 = Low
  2 = Moderate
  3 = Severe


In [174]:
y=df_encoded['Addiction_Level']
X=df_encoded.drop(columns=['Addiction_Level', 'Mental_Health_Score'])

In [167]:
X.head()

,Age,Gender,Academic_Level,Avg_Daily_Usage_Hours,Most_Used_Platform,Affects_Academic_Performance,Sleep_Hours_Per_Night,Mental_Health_Score,Relationship_Status,Conflicts_Over_Social_Media
0,19,0,2,5.2,1,1,6.5,6,1,3
1,22,1,0,2.1,7,0,7.5,8,2,0
2,20,0,2,6.0,6,1,5.0,5,0,4
3,18,1,1,3.0,11,0,7.0,7,2,1
4,21,1,0,4.5,0,1,6.0,6,1,2


In [152]:
y.value_counts()

Addiction_Level
0    353
2    280
3     55
1     17
Name: count, dtype: int64

In [153]:
y_total = len(y)
for label in sorted(y.unique()):
    ratio = (y == label).sum() / y_total
    print(f"{label} = {ratio:.2%}")

0 = 50.07%
1 = 2.41%
2 = 39.72%
3 = 7.80%


In [175]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)
clf = DecisionTreeClassifier(max_depth=5, min_samples_split=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

importances = pd.Series(clf.feature_importances_, index=X.columns)
print(importances.sort_values(ascending=False))

Accuracy: 0.8584905660377359
Precision: 0.8597534108612523
Recall: 0.8584905660377359
F1 Score: 0.8575934364495156
Affects_Academic_Performance    0.595135
Sleep_Hours_Per_Night           0.147580
Avg_Daily_Usage_Hours           0.112190
Most_Used_Platform              0.069141
Conflicts_Over_Social_Media     0.047083
Gender                          0.022176
Relationship_Status             0.006696
Age                             0.000000
Academic_Level                  0.000000
dtype: float64


In [155]:
print(dict(enumerate(le.classes_)))
print(y_train.value_counts())
print(y_test.value_counts())

{0: 'High', 1: 'Low', 2: 'Moderate', 3: 'Severe'}
Addiction_Level
0    239
2    200
3     42
1     12
Name: count, dtype: int64
Addiction_Level
0    114
2     80
3     13
1      5
Name: count, dtype: int64


In [ ]:
train_total = len(y_train)
test_total = len(y_test)

print(dict(enumerate(le.classes_)))

for label in sorted(y_train.unique()):
    train_ratio = (y_train == label).sum() / train_total
    test_ratio = (y_test == label).sum() / test_total
    print(f"{label} = Train: {train_ratio:.2%}, Test: {test_ratio:.2%}")

0 = Train: 48.48%, Test: 53.77%
1 = Train: 2.43%, Test: 2.36%
2 = Train: 40.57%, Test: 37.74%
3 = Train: 8.52%, Test: 6.13%


In [176]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0, stratify=y)
clf = DecisionTreeClassifier(max_depth=5, min_samples_split=10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

importances = pd.Series(clf.feature_importances_, index=X.columns)
print(importances.sort_values(ascending=False))

Accuracy: 0.8867924528301887
Precision: 0.8938570856852233
Recall: 0.8867924528301887
F1 Score: 0.8823480683779144
Affects_Academic_Performance    0.640403
Avg_Daily_Usage_Hours           0.106294
Sleep_Hours_Per_Night           0.079838
Most_Used_Platform              0.078726
Age                             0.032822
Gender                          0.027775
Conflicts_Over_Social_Media     0.026424
Relationship_Status             0.007717
Academic_Level                  0.000000
dtype: float64


In [158]:
print(dict(enumerate(le.classes_)))
print(y_train.value_counts())
print(y_test.value_counts())

{0: 'High', 1: 'Low', 2: 'Moderate', 3: 'Severe'}
Addiction_Level
0    247
2    196
3     38
1     12
Name: count, dtype: int64
Addiction_Level
0    106
2     84
3     17
1      5
Name: count, dtype: int64


In [ ]:
train_total = len(y_train)
test_total = len(y_test)

print(dict(enumerate(le.classes_)))

for label in sorted(y_train.unique()):
    train_ratio = (y_train == label).sum() / train_total
    test_ratio = (y_test == label).sum() / test_total
    print(f"{label} = Train: {train_ratio:.2%}, Test: {test_ratio:.2%}")

0 = Train: 50.10%, Test: 50.00%
1 = Train: 2.43%, Test: 2.36%
2 = Train: 39.76%, Test: 39.62%
3 = Train: 7.71%, Test: 8.02%


In [184]:
kfold_validation = KFold(n_splits=10, shuffle=True, random_state=42)
clf = DecisionTreeClassifier(max_depth=5, min_samples_split=10)

scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='weighted', zero_division=0),
    'recall': make_scorer(recall_score, average='weighted', zero_division=0),
    'f1': make_scorer(f1_score, average='weighted', zero_division=0)
}
results = cross_validate(clf, X, y, cv=kfold_validation, scoring=scoring)

for i in range(len(results['test_accuracy'])):
    print(f"Fold {i+1}: Accuracy={results['test_accuracy'][i]:.4f}, Precision={results['test_precision'][i]:.4f}, Recall={results['test_recall'][i]:.4f}, F1={results['test_f1'][i]:.4f}")

print("Average Scores:")
print(f"Mean Accuracy: {np.mean(results['test_accuracy']):.4f} +/- {np.std(results['test_accuracy']):.4f}")
print(f"Mean Precision: {np.mean(results['test_precision']):.4f} +/- {np.std(results['test_precision']):.4f}")
print(f"Mean Recall: {np.mean(results['test_recall']):.4f} + {np.std(results['test_recall']):.4f}")
print(f"Mean F1 Score: {np.mean(results['test_f1']):.4f} +/- {np.std(results['test_f1']):.4f}")

Fold 1: Accuracy=0.8873, Precision=0.8896, Recall=0.8873, F1=0.8874
Fold 2: Accuracy=0.9437, Precision=0.9464, Recall=0.9437, F1=0.9319
Fold 3: Accuracy=0.9296, Precision=0.9380, Recall=0.9296, F1=0.9274
Fold 4: Accuracy=0.9296, Precision=0.9331, Recall=0.9296, F1=0.9295
Fold 5: Accuracy=0.8873, Precision=0.8979, Recall=0.8873, F1=0.8857
Fold 6: Accuracy=0.8857, Precision=0.8910, Recall=0.8857, F1=0.8859
Fold 7: Accuracy=0.8571, Precision=0.8778, Recall=0.8571, F1=0.8594
Fold 8: Accuracy=0.9143, Precision=0.9165, Recall=0.9143, F1=0.9146
Fold 9: Accuracy=0.9143, Precision=0.9189, Recall=0.9143, F1=0.9108
Fold 10: Accuracy=0.9286, Precision=0.9280, Recall=0.9286, F1=0.9257
Average Scores:
Mean Accuracy: 0.9077 +/- 0.0258
Mean Precision: 0.9137 +/- 0.0222
Mean Recall: 0.9077 + 0.0258
Mean F1 Score: 0.9058 +/- 0.0234


In [185]:
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
clf = DecisionTreeClassifier(max_depth=5, min_samples_split=10)

scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='weighted', zero_division=0),
    'recall': make_scorer(recall_score, average='weighted', zero_division=0),
    'f1': make_scorer(f1_score, average='weighted', zero_division=0)
}
results = cross_validate(clf, X, y, cv = skfold, scoring=scoring)

for i in range(len(results['test_accuracy'])):
    print(f"Fold {i+1}: Accuracy={results['test_accuracy'][i]:.4f}, Precision={results['test_precision'][i]:.4f}, Recall={results['test_recall'][i]:.4f}, F1={results['test_f1'][i]:.4f}")

print("Average Scores:")
print(f"Mean Accuracy: {np.mean(results['test_accuracy']):.4f} +/- {np.std(results['test_accuracy']):.4f}")
print(f"Mean Precision: {np.mean(results['test_precision']):.4f} +/- {np.std(results['test_precision']):.4f}")
print(f"Mean Recall: {np.mean(results['test_recall']):.4f} + {np.std(results['test_recall']):.4f}")
print(f"Mean F1 Score: {np.mean(results['test_f1']):.4f} +/- {np.std(results['test_f1']):.4f}")

Fold 1: Accuracy=0.9437, Precision=0.9493, Recall=0.9437, F1=0.9433
Fold 2: Accuracy=0.9155, Precision=0.9215, Recall=0.9155, F1=0.9139
Fold 3: Accuracy=0.8873, Precision=0.8911, Recall=0.8873, F1=0.8860
Fold 4: Accuracy=0.8732, Precision=0.8989, Recall=0.8732, F1=0.8704
Fold 5: Accuracy=0.9155, Precision=0.9175, Recall=0.9155, F1=0.9154
Fold 6: Accuracy=0.9286, Precision=0.9375, Recall=0.9286, F1=0.9262
Fold 7: Accuracy=0.8429, Precision=0.8486, Recall=0.8429, F1=0.8421
Fold 8: Accuracy=0.9000, Precision=0.9016, Recall=0.9000, F1=0.8999
Fold 9: Accuracy=0.8857, Precision=0.8896, Recall=0.8857, F1=0.8837
Fold 10: Accuracy=0.9000, Precision=0.9107, Recall=0.9000, F1=0.8998
Average Scores:
Mean Accuracy: 0.8992 +/- 0.0275
Mean Precision: 0.9066 +/- 0.0267
Mean Recall: 0.8992 + 0.0275
Mean F1 Score: 0.8981 +/- 0.0276


In [186]:
ssplit = ShuffleSplit(n_splits = 10, test_size = 0.30, random_state = 42)
clf = DecisionTreeClassifier(max_depth=5, min_samples_split=10)

scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='weighted', zero_division=0),
    'recall': make_scorer(recall_score, average='weighted', zero_division=0),
    'f1': make_scorer(f1_score, average='weighted', zero_division=0)
}
results = cross_validate(clf, X, y, cv = ssplit, scoring = scoring)

for i in range(len(results['test_accuracy'])):
    print(f"Fold {i+1}: Accuracy={results['test_accuracy'][i]:.4f}, Precision={results['test_precision'][i]:.4f}, Recall={results['test_recall'][i]:.4f}, F1={results['test_f1'][i]:.4f}")
    
print("Average Scores:")
print(f"Mean Accuracy: {np.mean(results['test_accuracy']):.4f} +/- {np.std(results['test_accuracy']):.4f}")
print(f"Mean Precision: {np.mean(results['test_precision']):.4f} +/- {np.std(results['test_precision']):.4f}")
print(f"Mean Recall: {np.mean(results['test_recall']):.4f} + {np.std(results['test_recall']):.4f}")
print(f"Mean F1 Score: {np.mean(results['test_f1']):.4f} +/- {np.std(results['test_f1']):.4f}")

Fold 1: Accuracy=0.9198, Precision=0.9309, Recall=0.9198, F1=0.9175
Fold 2: Accuracy=0.9245, Precision=0.9263, Recall=0.9245, F1=0.9245
Fold 3: Accuracy=0.9245, Precision=0.9282, Recall=0.9245, F1=0.9236
Fold 4: Accuracy=0.8538, Precision=0.8821, Recall=0.8538, F1=0.8504
Fold 5: Accuracy=0.8915, Precision=0.8977, Recall=0.8915, F1=0.8934
Fold 6: Accuracy=0.8585, Precision=0.8590, Recall=0.8585, F1=0.8566
Fold 7: Accuracy=0.9104, Precision=0.9184, Recall=0.9104, F1=0.9111
Fold 8: Accuracy=0.9104, Precision=0.9194, Recall=0.9104, F1=0.9103
Fold 9: Accuracy=0.9198, Precision=0.9215, Recall=0.9198, F1=0.9188
Fold 10: Accuracy=0.8915, Precision=0.8925, Recall=0.8915, F1=0.8896
Average Scores:
Mean Accuracy: 0.9005 +/- 0.0249
Mean Precision: 0.9076 +/- 0.0226
Mean Recall: 0.9005 + 0.0249
Mean F1 Score: 0.8996 +/- 0.0256
